In [3]:
from config import hdfs_config

In [4]:
HDFS_HOST = hdfs_config.HDFSConfig.HOST
HDFS_PORT = hdfs_config.HDFSConfig.PORT
LOCAL_RAW_DATA_PATH = hdfs_config.HDFSConfig.LOCAL_RAW_DATA_PATH
HDFS_RAW_DEST_PATH = hdfs_config.HDFSConfig.RAW_DEST_PATH
HDFS_CLEAN_DEST_PATH = hdfs_config.HDFSConfig.CLEAN_DEST_PATH
LOCAL_CLEAN_DATA_PATH = hdfs_config.HDFSConfig.LOCAL_CLEAN_DATA_PATH



In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col
import pandas as pd
import re
import os

# Initialize Spark session
def initialize_spark():
    return SparkSession.builder.appName("Data Cleaning").getOrCreate()

# Extract quarter start and end dates from Quarterly Activity sheet (specific to FFEL Dashboard files)
def extract_quarter_dates(file_path):
    try:
        metadata_df = pd.read_excel(file_path, sheet_name='Quarterly Activity', nrows=5)
        metadata_text = " ".join(metadata_df.astype(str).values.flatten())
        quarter_dates_match = re.search(r"\((\d{2}/\d{2}/\d{4})-(\d{2}/\d{2}/\d{4})\)", metadata_text)
        quarter_start = quarter_dates_match.group(1) if quarter_dates_match else None
        quarter_end = quarter_dates_match.group(2) if quarter_dates_match else None
        return quarter_start, quarter_end
    except Exception as e:
        print(f"Error extracting quarter dates: {e}")
        return None, None

# Load FFEL Dashboard data from Excel sheets and convert to Spark DataFrames
def load_ffel_data(file_path, spark):
    try:
        quarterly_activity_pd = pd.read_excel(file_path, sheet_name='Quarterly Activity', skiprows=5, dtype={'OPE ID': str})
        award_year_summary_pd = pd.read_excel(file_path, sheet_name='Award Year Summary', skiprows=5, dtype={'OPE ID': str})
        quarterly_activity_spark = spark.createDataFrame(quarterly_activity_pd)
        award_year_summary_spark = spark.createDataFrame(award_year_summary_pd)
        return quarterly_activity_spark, award_year_summary_spark
    except Exception as e:
        print(f"Error loading data from {file_path}: {e}")
        return None, None


# Add quarter dates for FFEL data
def add_quarter_dates(quarterly_df, award_year_df, quarter_start, quarter_end):
    quarterly_df = quarterly_df.withColumn("Quarter_Start", lit(quarter_start)).withColumn("Quarter_End", lit(quarter_end))
    award_year_df = award_year_df.withColumn("Quarter_Start", lit(None)).withColumn("Quarter_End", lit(None))
    return quarterly_df, award_year_df

# Combine DataFrames and remove duplicates
def combine_dataframes(*dfs):
    combined_df = dfs[0]
    for df in dfs[1:]:
        combined_df = combined_df.union(df)
    return combined_df.dropDuplicates()

# Rename columns to a standardized format
def rename_columns(df, column_mapping):
    for old_name, new_name in column_mapping.items():
        if old_name in df.columns:
            df = df.withColumnRenamed(old_name, new_name)
    return df

# Convert specified columns to double type
def cast_columns_to_double(df, columns_to_cast):
    for col_name in columns_to_cast:
        if col_name in df.columns:
            df = df.withColumn(col_name, col(col_name).cast("double"))
    return df

# Save the cleaned DataFrame to Excel
def save_to_excel(df, output_path):
    try:
        # Ensure output is saved in `.xlsx` format for compatibility
        if output_path.endswith(".xls"):
            output_path = output_path.replace(".xls", ".xlsx")
            print(f"Converted output file to .xlsx format: {output_path}")

        df_pd = df.toPandas()
        df_pd.to_excel(output_path, index=False, float_format="%.2f")
        print(f"Saved cleaned data to {output_path}")
    except Exception as e:
        print(f"Error saving to Excel: {e}")

# Main function to process FFEL data files
def process_ffel_data(file_path, output_path,spark):
    
    
    # Extract quarter dates
    quarter_start, quarter_end = extract_quarter_dates(file_path)
    
    # Load FFEL data
    quarterly_activity_spark, award_year_summary_spark = load_ffel_data(file_path, spark)
    if not quarterly_activity_spark or not award_year_summary_spark:
        print(f"Skipping {file_path} due to load errors.")
        return
    
    # Add quarter dates
    quarterly_activity_spark, award_year_summary_spark = add_quarter_dates(
        quarterly_activity_spark, award_year_summary_spark, quarter_start, quarter_end
    )
    
    # Combine data
    combined_df = combine_dataframes(quarterly_activity_spark, award_year_summary_spark)
    
    # Column renaming mappings
    column_mapping = {
        "# of Loans Originated": "ffel_subsidized_number_of_loans_originated",
        "$ of Loans Originated": "ffel_subsidized_amount_of_loans_originated",
        "Recipients": "ffel_subsidized_recipients",
        "# of Loans Originated.1": "ffel_unsubsidized_number_of_loans_originated",
        "$ of Loans Originated.1": "ffel_unsubsidized_amount_of_loans_originated",
        "Recipients.1": "ffel_unsubsidized_recipients",
        "# of Loans Originated.2": "ffel_stafford_number_of_loans_originated",
        "$ of Loans Originated.2": "ffel_stafford_amount_of_loans_originated",
        "Recipients.2": "ffel_stafford_recipients",
        "# of Loans Originated.3": "ffel_plus_number_of_loans_originated",
        "$ of Loans Originated.3": "ffel_plus_amount_of_loans_originated",
        "Recipients.3": "ffel_plus_recipients",
        "# of Disbursements": "ffel_subsidized_number_of_disbursements",
        "$ of Disbursements": "ffel_subsidized_amount_of_disbursements",
        "# of Disbursements.1": "ffel_unsubsidized_number_of_disbursements",
        "$ of Disbursements.1": "ffel_unsubsidized_amount_of_disbursements",
        "# of Disbursements.2": "ffel_stafford_number_of_disbursements",
        "$ of Disbursements.2": "ffel_stafford_amount_of_disbursements",
        "# of Disbursements.3": "ffel_plus_number_of_disbursements",
        "$ of Disbursements.3": "ffel_plus_amount_of_disbursements"
    }
    
    # Rename columns
    combined_df = rename_columns(combined_df, column_mapping)
    
    # Columns to cast to double
    columns_to_cast = [
        "ffel_subsidized_amount_of_loans_originated", "ffel_unsubsidized_amount_of_loans_originated",
        "ffel_stafford_amount_of_loans_originated", "ffel_plus_amount_of_loans_originated",
        "ffel_subsidized_amount_of_disbursements", "ffel_unsubsidized_amount_of_disbursements",
        "ffel_stafford_amount_of_disbursements", "ffel_plus_amount_of_disbursements"
    ]
    
    # Cast columns to double
    combined_df = cast_columns_to_double(combined_df, columns_to_cast)
    
    # Save to Excel
    save_to_excel(combined_df, output_path)
    
    



# Process all files in raw directory
def process_all_files(raw_dir, cleaned_dir):
    spark = initialize_spark()
    
    try:
        if not os.path.exists(cleaned_dir):
            os.makedirs(cleaned_dir)
        
        raw_files = [f for f in os.listdir(raw_dir) if f.endswith(".xls") or f.endswith(".xlsx")]
        
        for file_name in raw_files:
            raw_file_path = os.path.join(raw_dir, file_name)
            cleaned_file_path = os.path.join(cleaned_dir, f"cleaned_{file_name}")
            
            try:
                process_ffel_data(raw_file_path, cleaned_file_path,spark)
            except Exception as e:
                print(f"Error processing {file_name}: {e}")
    finally:
        # Stop Spark session
        spark.stop()




# Load Fed School Code data and convert to Spark DataFrame
def load_fed_school_code_data(file_path, spark):
    # Customize based on the structure of the Fed School Code file
    fed_school_code_pd = pd.read_excel(file_path)  # Adjust sheet name if different
    fed_school_code_spark = spark.createDataFrame(fed_school_code_pd)
    
    return fed_school_code_spark


# Main function to process Fed School Code data files
def process_fed_school_code_data(file_path, output_path):
    spark = initialize_spark()
    try:
        # Load Fed School Code data
        fed_school_code_spark = load_fed_school_code_data(file_path, spark)
        
        # Save to Excel
        save_to_excel(fed_school_code_spark, output_path)
    
    finally:
        # Stop Spark session
        spark.stop()


file_path = '/home/freddy/Documents/Cours_efrei/Data_integration/Projets_data_integration/data_integration_student_loans/data/raw/'
output_path ='/home/freddy/Documents/Cours_efrei/Data_integration/Projets_data_integration/data_integration_student_loans/data/cleaned'
fedschool_path ='/home/freddy/Documents/Cours_efrei/Data_integration/Projets_data_integration/data_integration_student_loans/data/1617fedschoolcodelist.xls'
fedschooloutput_path ='/home/freddy/Documents/Cours_efrei/Data_integration/Projets_data_integration/data_integration_student_loans/datacleaned_Fed_School_Code_data.xlsx'
# process_ffel_data(file_path,output_path)

process_all_files(file_path, output_path)

24/11/17 13:09:05 WARN Utils: Your hostname, freddy-Nitro-AN515-55 resolves to a loopback address: 127.0.1.1; using 192.168.1.223 instead (on interface wlp0s20f3)
24/11/17 13:09:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/17 13:09:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/17 13:09:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Saved cleaned data to /home/freddy/Documents/Cours_efrei/Data_integration/Projets_data_integration/data_integration_student_loans/data/cleaned/cleaned_FL_Dashboard_AY2009_2010_Q1.xlsx


24/11/17 13:09:19 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Saved cleaned data to /home/freddy/Documents/Cours_efrei/Data_integration/Projets_data_integration/data_integration_student_loans/data/cleaned/cleaned_FL_Dashboard_AY2009_2010_Q2.xlsx
Converted output file to .xlsx format: /home/freddy/Documents/Cours_efrei/Data_integration/Projets_data_integration/data_integration_student_loans/data/cleaned/cleaned_FL_Dashboard_AY2009_2010_Q3.xlsx


Saved cleaned data to /home/freddy/Documents/Cours_efrei/Data_integration/Projets_data_integration/data_integration_student_loans/data/cleaned/cleaned_FL_Dashboard_AY2009_2010_Q3.xlsx
Converted output file to .xlsx format: /home/freddy/Documents/Cours_efrei/Data_integration/Projets_data_integration/data_integration_student_loans/data/cleaned/cleaned_FL_Dashboard_AY2009_2010_Q4.xlsx


Saved cleaned data to /home/freddy/Documents/Cours_efrei/Data_integration/Projets_data_integration/data_integration_student_loans/data/cleaned/cleaned_FL_Dashboard_AY2009_2010_Q4.xlsx


In [6]:
import os
import pyhdfs
import logging
from config import hdfs_config


def upload_files_to_hdfs(local_path, hdfs_path, hdfs_client):
    """Uploads files from a local directory to HDFS with detailed error handling."""
    try:
        # Ensure the HDFS directory exists
        if not hdfs_client.exists(hdfs_path):
            hdfs_client.mkdirs(hdfs_path)
            logging.info(f"Created HDFS directory: {hdfs_path}")
        
        # Check if the local directory exists
        if not os.path.exists(local_path):
            logging.error(f"Local directory {local_path} does not exist.")
            return

        # Iterate through files in the local raw data directory
        for file_name in os.listdir(local_path):
            local_file_path = os.path.join(local_path, file_name)

            # Only process files
            if os.path.isfile(local_file_path):
                hdfs_file_path = f"{hdfs_path}/{file_name}"

                # Check if the file already exists in HDFS
                if hdfs_client.exists(hdfs_file_path):
                    logging.info(f"File {hdfs_file_path} already exists in HDFS. Skipping upload.")
                    continue

                # Attempt to upload file to HDFS
                try:
                    with open(local_file_path, 'rb') as file_data:
                        hdfs_client.create(hdfs_file_path, file_data)
                    logging.info(f"Uploaded {file_name} to HDFS at {hdfs_file_path}")

                    # Verify upload success
                    if hdfs_client.exists(hdfs_file_path):
                        logging.info(f"Successfully uploaded: {file_name}")
                    else:
                        logging.error(f"Upload verification failed: {file_name}")
                except Exception as e:
                    logging.error(f"Failed to upload {file_name} to HDFS due to {e}")

            else:
                logging.warning(f"{local_file_path} is not a file. Skipping.")

    except Exception as e:
        logging.error(f"Unexpected error during file upload: {e}")

In [8]:
try:
    # Initialize HDFS client
    hdfs_client = pyhdfs.HdfsClient(hosts=f"{HDFS_HOST}:{HDFS_PORT}")
    logging.info("HDFS client initialized")
    
    # Start uploading files
    upload_files_to_hdfs(LOCAL_RAW_DATA_PATH, HDFS_RAW_DEST_PATH, hdfs_client)
    upload_files_to_hdfs(LOCAL_CLEAN_DATA_PATH, HDFS_CLEAN_DEST_PATH, hdfs_client)
    
    
except pyhdfs.HdfsException as he:
    logging.error(f"HDFS error: Failed to initialize HDFS client or perform operations due to {he}")
except Exception as e:
    logging.error(f"Unexpected error: {e}")


In [10]:
from hdfs import InsecureClient
import pandas as pd
from io import BytesIO

def read_files_from_hdfs(hdfs_directory_path, hdfs_host, hdfs_port, nrows=None):
    """
    Reads each file in the specified HDFS directory into a separate DataFrame.
    
    Parameters:
        hdfs_directory_path (str): HDFS directory path containing files to read.
        hdfs_host (str): HDFS host address.
        hdfs_port (int): HDFS port number.
        nrows (int, optional): Number of rows to read from each file (for memory efficiency).
    
    Returns:
        dict: A dictionary with file names as keys and DataFrames as values.
    """
    client = InsecureClient(f'http://{hdfs_host}:{hdfs_port}')
    data_frames = {}
    
    try:
        # List all files in the specified HDFS directory
        files = client.list(hdfs_directory_path)
        
        for file_name in files:
            file_path = f"{hdfs_directory_path}/{file_name}"
            
            # Read file content from HDFS
            with client.read(file_path) as f:
                file_content = f.read()
            
            # Use BytesIO for compatibility with pandas
            file_data = BytesIO(file_content)
            
            # Load the data into a DataFrame and store it in the dictionary
            data_frames[file_name] = pd.read_excel(file_data, nrows=nrows)
            print(f"Loaded file '{file_name}' into a DataFrame.")
        
        return data_frames
    
    except Exception as e:
        print(f"Error reading files from HDFS: {e}")
        return None

# Example usage
data_frames = read_files_from_hdfs(HDFS_CLEAN_DEST_PATH, HDFS_HOST, HDFS_PORT, nrows=None)


Loaded file 'cleaned_FL_Dashboard_AY2009_2010_Q1.xlsx' into a DataFrame.
Loaded file 'cleaned_FL_Dashboard_AY2009_2010_Q2.xlsx' into a DataFrame.
Loaded file 'cleaned_FL_Dashboard_AY2009_2010_Q3.xlsx' into a DataFrame.
Loaded file 'cleaned_FL_Dashboard_AY2009_2010_Q4.xlsx' into a DataFrame.


In [11]:
for file_name, df in data_frames.items():
    print(f"Cleaning DataFrame for file: {file_name}")
    # Perform individual cleaning steps for each DataFrame
    # Example: df = df.dropna() or any other cleaning function you wish to apply
    # After cleaning, you can save or process each df as needed

    # Example: Show the first few rows of each DataFrame
    print(df)

Cleaning DataFrame for file: cleaned_FL_Dashboard_AY2009_2010_Q1.xlsx
       OPE ID                                       School State   Zip Code  \
0     2612800                 LOS ANGELES RECORDING SCHOOL    CA  900283112   
1     3113600  SOUTHERN CALIFORNIA INSTITUTE OF TECHNOLOGY    CA  928053758   
2      718700                NORTH ADRIAN'S BEAUTY COLLEGE    CA  953502381   
3     2619300        SAMRA UNIVERSITY OF ORIENTAL MEDICINE    CA  900151019   
4     2561900                BRILLARE HAIRDRESSING ACADEMY    AZ  852543301   
...       ...                                          ...   ...        ...   
7635  2305300                            PARKER UNIVERSITY    TX  752295609   
7636  2591100                         CAREER POINT COLLEGE    TX  782012027   
7637  3416500                     DALLAS NURSING INSTITUTE    TX  752435502   
7638   367200           LATTER DAY SAINTS BUSINESS COLLEGE    UT  841013500   
7639  1036400                    WHATCOM COMMUNITY COLLEGE   

In [18]:
import os
import time
import json
import pandas as pd
from kafka import KafkaProducer
from config.kafka_config import KAFKA_BROKER_URL, TOPIC_NAME_FL_DASHBOARD, BATCH_SIZE, POLL_INTERVAL

# Initialize Kafka producer
producer = KafkaProducer(
    bootstrap_servers=KAFKA_BROKER_URL,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')  # Serialize data to JSON
)

def read_data_in_batches(file_path, batch_size):
    """Reads data from file in batches."""
    """Reads data from an Excel file in batches."""
    df = pd.read_excel(file_path)
    for start in range(0, len(df), batch_size):
        yield df[start:start + batch_size].to_dict(orient='records')

def send_data_to_kafka(topic, file_path):
    """Sends data from the file to Kafka in batches."""
    for batch in read_data_in_batches(file_path, BATCH_SIZE):
        for record in batch:
            producer.send(topic, value=record)  # Send each record to Kafka
        producer.flush()  # Ensure data is sent before sleeping
        print(f"Sent batch of {len(batch)} records to Kafka topic '{topic}'")
        time.sleep(POLL_INTERVAL)

def main():
    # Path to raw data files
    raw_data_dir = os.path.join("data", "raw")
    
    # Send FL_Dashboard files
    for filename in os.listdir(raw_data_dir):
        if filename.startswith("FL_Dashboard"):
            file_path = os.path.join(raw_data_dir, filename)
            print(f"Streaming data from file: {file_path}")
            send_data_to_kafka(TOPIC_NAME_FL_DASHBOARD, file_path)

In [ ]:
main()
producer.close()

Streaming data from file: data/raw/FL_Dashboard_AY2009_2010_Q1.xlsx
Sent batch of 100 records to Kafka topic 'fl_dashboard_topic'
Sent batch of 100 records to Kafka topic 'fl_dashboard_topic'
Sent batch of 100 records to Kafka topic 'fl_dashboard_topic'
Sent batch of 100 records to Kafka topic 'fl_dashboard_topic'
Sent batch of 100 records to Kafka topic 'fl_dashboard_topic'
Sent batch of 100 records to Kafka topic 'fl_dashboard_topic'
Sent batch of 100 records to Kafka topic 'fl_dashboard_topic'
Sent batch of 100 records to Kafka topic 'fl_dashboard_topic'
Sent batch of 100 records to Kafka topic 'fl_dashboard_topic'
Sent batch of 100 records to Kafka topic 'fl_dashboard_topic'
Sent batch of 100 records to Kafka topic 'fl_dashboard_topic'
Sent batch of 100 records to Kafka topic 'fl_dashboard_topic'
Sent batch of 100 records to Kafka topic 'fl_dashboard_topic'
Sent batch of 100 records to Kafka topic 'fl_dashboard_topic'
Sent batch of 100 records to Kafka topic 'fl_dashboard_topic'
Se

In [ ]:
# Kafka consumer to pull data from Kafka

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from config.kafka_config import KAFKA_BROKER_URL, TOPIC_NAME_FL_DASHBOARD, TOPIC_NAME_SCHOOL_CODELIST
from config.spark_config import APP_NAME, MASTER, BATCH_DURATION
import json

def start_spark_streaming():
    # Initialize Spark session
    spark = SparkSession.builder \
        .appName(APP_NAME) \
        .master(MASTER) \
        .getOrCreate()

    spark.sparkContext.setLogLevel("WARN")

    # Read from Kafka
    df_kafka = spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", KAFKA_BROKER_URL) \
        .option("subscribe", f"{TOPIC_NAME_FL_DASHBOARD},{TOPIC_NAME_SCHOOL_CODELIST}") \
        .option("startingOffsets", "earliest") \
        .load()

    # Convert Kafka's binary data to string
    df_kafka = df_kafka.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

    # Define schema or processing functions as needed for incoming data
    # This example assumes JSON strings; adjust according to your data format
    schema = "your_schema_here"  # Define your schema here

    df_parsed = df_kafka.withColumn("value", from_json(col("value"), schema))

    # Write the parsed data to the console (for debugging)
    query = df_parsed.writeStream \
        .outputMode("append") \
        .format("console") \
        .start()

    # Wait for the termination signal
    query.awaitTermination()


start_spark_streaming()

localhost
